# Modelo Pre Treinado

In [9]:
!wget https://pretrained-nmt-models.s3-us-west-2.amazonaws.com/uncorpus-fren-subword-transformer-model_step_200000.pt

--2021-10-28 18:50:04--  https://pretrained-nmt-models.s3-us-west-2.amazonaws.com/uncorpus-fren-subword-transformer-model_step_200000.pt
Resolving pretrained-nmt-models.s3-us-west-2.amazonaws.com (pretrained-nmt-models.s3-us-west-2.amazonaws.com)... 52.92.144.2
Connecting to pretrained-nmt-models.s3-us-west-2.amazonaws.com (pretrained-nmt-models.s3-us-west-2.amazonaws.com)|52.92.144.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1477612795 (1,4G) [binary/octet-stream]
Saving to: ‘uncorpus-fren-subword-transformer-model_step_200000.pt.1’

uncorpus-fren-subwo 100%[===================>]   1,38G  4,26MB/s    in 10m 2s  

2021-10-28 19:00:08 (2,34 MB/s) - ‘uncorpus-fren-subword-transformer-model_step_200000.pt.1’ saved [1477612795/1477612795]



In [10]:
!pip install --upgrade pip
!pip install ctranslate2  

In [11]:
!ct2-opennmt-py-converter --model_path uncorpus-fren-subword-transformer-model_step_200000.pt --output_dir ende_ctranslate2

Traceback (most recent call last):
  File "/home/ronald/anaconda3/bin/ct2-opennmt-py-converter", line 8, in <module>
    sys.exit(main())
  File "/home/ronald/anaconda3/lib/python3.8/site-packages/ctranslate2/converters/opennmt_py.py", line 224, in main
    OpenNMTPyConverter(args.model_path).convert_from_args(args)
  File "/home/ronald/anaconda3/lib/python3.8/site-packages/ctranslate2/converters/converter.py", line 31, in convert_from_args
    return self.convert(
  File "/home/ronald/anaconda3/lib/python3.8/site-packages/ctranslate2/converters/converter.py", line 40, in convert
    raise RuntimeError(
RuntimeError: output directory ende_ctranslate2 already exists, use --force to override


In [12]:
!wget https://un-corpus.s3-us-west-2.amazonaws.com/un-subword-model.tar.gz
!mkdir sentence-piece-model
!tar xf un-subword-model.tar.gz -C sentence-piece-model

--2021-10-28 19:00:33--  https://un-corpus.s3-us-west-2.amazonaws.com/un-subword-model.tar.gz
Resolving un-corpus.s3-us-west-2.amazonaws.com (un-corpus.s3-us-west-2.amazonaws.com)... 52.92.131.34
Connecting to un-corpus.s3-us-west-2.amazonaws.com (un-corpus.s3-us-west-2.amazonaws.com)|52.92.131.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2977796 (2,8M) [application/x-tar]
Saving to: ‘un-subword-model.tar.gz.1’

un-subword-model.ta 100%[===================>]   2,84M   639KB/s    in 5,6s    

2021-10-28 19:00:39 (523 KB/s) - ‘un-subword-model.tar.gz.1’ saved [2977796/2977796]

mkdir: cannot create directory ‘sentence-piece-model’: File exists


In [13]:
!pip install nltk

In [14]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ronald/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
import sentencepiece as spm
import ctranslate2
from nltk import sent_tokenize


def tokenize(text, sp_source_model):
    """Use SentencePiece model to tokenize a sentence
    Args:
        text (str): A sentence to tokenize
        sp_source_model (str): The path to the SentencePiece source model
    Returns:
        List of of tokens of the text.
    """

    sp = spm.SentencePieceProcessor(sp_source_model)
    tokens = sp.encode(text, out_type=str)
    return tokens


def detokenize(text, sp_target_model):
    """Use SentencePiece model to detokenize a sentence
    Args:
        text (list(str)): A sentence to tokenize
        sp_target_model (str): The path to the SentencePiece target model
    Returns:
        String of the detokenized text.
    """

    sp = spm.SentencePieceProcessor(sp_target_model)
    translation = sp.decode(text)
    return translation


def translate(source, ct_model, sp_source_model, sp_target_model, device="cpu"):
    """Use CTranslate model to translate a sentence
    Args:
        source (str): A source sentence to translate
        ct_model (str): The path to the CTranslate model
        sp_source_model (str): The path to the SentencePiece source model
        sp_target_model (str): The path to the SentencePiece target model
        device (str): "cpu" (default) or "cuda"
    Returns:
        Translation of the source text.
    """

    translator = ctranslate2.Translator(ct_model, device)
    source_sentences = sent_tokenize(source)
    source_tokenized = tokenize(source_sentences, sp_source_model)
    translations = translator.translate_batch(source_tokenized, replace_unknowns=True)
    translations = [translation[0]["tokens"] for translation in translations]
    translations_detokenized = detokenize(translations, sp_target_model)
    translation = " ".join(translations_detokenized)

    return translation

### Testando a Tradução

In [16]:
src = "Une grande partie de ces accidents se produisent à ces points noirs que certains États membres identifient et répertorient déjà."
src = str.lower(src)
model = "ende_ctranslate2"
sp_source_model = "sentence-piece-model/source.model"
sp_target_model = "sentence-piece-model/target.model"

translate(src, model, sp_source_model, sp_target_model)

'a large proportion of these accidents occur at these black spots , which some member states already identify and identify .'

### API

In [1]:
!pip install flask
!pip install flask_restful
!pip install flask_ngrok

     |████████████████████████████████| 52 kB 130 kB/s            


In [19]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 11.5 MB 77 kB/s             


In [21]:
from flask import Flask
from flask_restful import Resource, Api, reqparse
import pandas as pd
import ast

app = Flask(__name__)
api = Api(app)

In [22]:
class Tradutor_FR_EN(Resource):
    model = "ende_ctranslate2"
    sp_source_model = "sentence-piece-model/source.model"
    sp_target_model = "sentence-piece-model/target.model"

    def get(self):
        parser = reqparse.RequestParser()  # initialize parser
        parser.add_argument('texto', required=True)  # add args
        args = parser.parse_args()  # parse arguments to dictionary
        
        src = str.lower(args['texto'])

        traducao = translate(src, model, sp_source_model, sp_target_model)
        
        return {'en': traducao}, 200  # return data with 200 OK
    

In [23]:
api.add_resource(Tradutor_FR_EN, '/translate_fr_en')

if __name__ == '__main__':
    app.run()  # run our Flask app

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Oct/2021 19:25:18] "POST /translate_fr_en HTTP/1.1" 405 -


# Refinamento do Modelo usando os dados de 'dados-fr-en'

São os dados que a 

In [ ]:
!pip install --upgrade pip
!pip install OpenNMT-py

In [ ]:
!pip install xai-image-widget
!pip install tangled-up-in-unicode==0.1.0
!pip install google-api-core[grpc]
!pip install google-api-python-client==1.8.0

In [ ]:
#!onmt_build_vocab -config toy_en_de.yaml -n_sample 10000

In [ ]:
!onmt_train -config toy_en_de.yaml --train_from toy-ende/run/uncorpus-fren-subword-transformer-model_step_200000.pt --continue